<a href="https://colab.research.google.com/github/mrphys/UCL_ICS_researchProjects/blob/master/MatchingAlgorithmFor_ICSresearchProjects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code used for matching UCL ICS students to research projects for the iBSc and MSc for 2022

Here we use a mathematical optimizer to 'best' match between the students choices to the research projects available.

We have used Google's open source software suite for optimization, OR-Tools, which provides an MPSolver wrapper for solving linear programming and mixed integer programming problems.

This is a mixed integer programming (MIP) problem
Since the constraints are linear, this is  a linear optimization problem in which the solutions are required to be integers. 

More information about algorithm can be found here:
https://developers.google.com/optimization/mip/mip_example

We use the MIP solver 'SCIP' (Solving constraint integer problems). https://www.scipopt.org which is  currently one of the fastest non-commercial solvers for mixed integer programming.

SCIP is a framework for Constraint Integer Programming oriented towards the needs of mathematical programming experts who want to have total control of the solution process and access detailed information down to the guts of the solver. 

In SCIP the problem is successively divided into smaller subproblems (branching) that are solved recursively.(details of the algorithm can be found here: https://www.scipopt.org/doc/html/WHATPROBLEMS.php )

In [1]:
#Import some libraries

!pip install ortools
from ortools.linear_solver import pywraplp

import numpy as np 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


THIS PART IS WHERE THE STUDENT IDs ARE ENTERED FOR THE COHORT AS WELL AS THEIR PROJECT CHOICES.

Note: This needs editing for each new cohort/run

In [2]:
# These are the Student ID's: one ID per student (in reality we use the UCL ID)

studentIDs = [
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
]
num_students = len(studentIDs)

#These are the Students project choices (the number represets the project #ID). There should be one row per student above. Each student can have a different number of projects in their selection
#This example is based on the student selections from 2021/22
#Note: For 2022/23 students will have to select a minimum of 5 projects

projectChoices = [
  [69,	70,	64],
  [21,	36,	69],
  [12,	64,	65, 1, 2],
  [13,	30,	32],
  [20,	21,	61, 7],
  [60,	29,	71],
  [34,	3,	36],
  [20,	13,	39],
  [69,	65,	64],
  [21,	36,	4],
  [2,	  45,	21],
  [21,	20,	32],
  [13,	37,	60],
  [68,	37, 69,	46],
  [13,	53,	61],
  [3,	  69,	46],
  [21,	20,	67],
  [30,	32,	31],
  [35,	45,	34],
  [1,	  59,	68],
  [21,	68, 12,	30],
  [6,	  21,	26],
  [69,	12,	36],
  [35,	34,	36],
  [46,	21,	20],
  [28,	32,	61, 69],
  [61,	13,	37],
  [36,	4,	2, 20],
  [32,	70,	72],
  [68,	13,	69],
  [36,	10,	13],
  [1,	  70, 10,	12],
  [37,	30,	31],
]

# Here we do some checks on the data
if( len(projectChoices) != num_students):
  print("SOMETHING HAS GONE WRONG AS SIZE OF PROJECT CHOICES IS NOT EQUAL TO NUMBER OF STUDENTS!!!!")
  
  raise RuntimeError('SOMETHING HAS GONE WRONG AS SIZE OF PROJECT CHOICES IS NOT EQUAL TO NUMBER OF STUDENTS!!!!')

else:
  print("\n\n ALL OK TO CONTINUE: num_students = ", num_students, ", Number of students in selection = ", len(projectChoices))




 ALL OK TO CONTINUE: num_students =  33 , Number of students in selection =  33


In [3]:
# check no student has put less than 3 choices in!!

minimumAllowedProjectSelected = 3

for s in range(num_students):
  currentSelection = projectChoices[s]

  if len(currentSelection) < minimumAllowedProjectSelected:
    print("WARNING: student ", s+1, " (", studentIDs[s], ") has not selected more than ", minimumAllowedProjectSelected, " projects! CHECK BEOFRE YOU CONTINUE")  

    raise RuntimeError('SOMETHING HAS GONE WRONG: student ', s+1, ' (', studentIDs[s], ') has not selected more than ', minimumAllowedProjectSelected, ' projects! CHECK BEOFRE YOU CONTINUE')

print("\n\n ALL OK TO CONTINUE: no projects are selected more than once by the same student")    



 ALL OK TO CONTINUE: no projects are selected more than once by the same student


In [4]:
# check no student has put the same project number down more than once!

for s in range(num_students):
  currentSelection = projectChoices[s]

  bDuplicatedProjects = 0
  for p in range(len(currentSelection)):
      for p2 in range(p-1):
        if currentSelection[p] == currentSelection[p2]:
          bDuplicatedProjects = 1

  if bDuplicatedProjects == 1:
    print("WARNING: student ", s+1, " (", studentIDs[s], ") has duplicated projects! CHECK BEOFRE YOU CONTINUE")  

    raise RuntimeError('SOMETHING HAS GONE WRONG: student ',s+1,' (ID: ',int(studentIDs[s]),') has duplicated projects! CHECK BEOFRE YOU CONTINUE') 

print("\n\n ALL OK TO CONTINUE: no projects are selected more than once by the same student")       



 ALL OK TO CONTINUE: no projects are selected more than once by the same student


In [5]:
# For each of the projects proposed on the list, the supervisor specifies the maximum students they are able to take on this project
# This information is used by the algorithm to ensure that some projects can be allocated to more than one student!

#These are in the same order as the project numbers, so the first row is project #1, and the second row project #2, etc.

maxNoStudentsPerProj = [
  2,
  2,
  2,
  2,
  2,
  3,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  1,
  1,
  1,
]
totalNoProjectsOffered = len(maxNoStudentsPerProj)

In [6]:
# check no student has put the a poject number greater than the number of projects!

maxProjVal = 0
for s in range(num_students):
  currentSelection = projectChoices[s]

  if(max(currentSelection) > maxProjVal ):
    maxProjVal = max(currentSelection)

if totalNoProjectsOffered < maxProjVal:
  print("SOMETHING HAS GONE WRONG AS MAXIMUM SELECTED PROJECT NUMBER IS HIGHER THAN NUMBER OF PROJECTS!!!!")

  raise RuntimeError('SOMETHING HAS GONE WRONG: maximum project number selected by a student is ',maxProjVal,' but total Number of Projects offered is ', totalNoProjectsOffered, '! CHECK BEOFRE YOU CONTINUE') 

else:
  print("\n\n ALL OK TO CONTINUE: totalNoProjects = ", totalNoProjectsOffered, ", highest project number selected by any student = ", maxProjVal)




 ALL OK TO CONTINUE: totalNoProjects =  72 , highest project number selected by any student =  72


In [7]:
# Create a 'weighing scheme' for the project selection
## This weighting could be adapted to more heavily weight lower ranked projects
weighting_1stChoice = 0
weighting_2ndChoice = 1
weighting_3rdChoice = 2
weighting_4thChoice = 3
weighting_5thChoice = 4
weighting_6thChoice = 5
weighting_7thChoice = 6
weighting_8thChoice = 7
weighting_other = 100

# As this is a miminimzation problem, the higher ranked a project is then the lower the 'weighting'. 

# Then we creates an array which is size: [number of students x total number of projects]
# This array is filled with value '100' from weighting_other, i.e. no students want any projects
costs = np.full((num_students, totalNoProjectsOffered), weighting_other, dtype=int)

# Then for each student:
# the first choice project position in this array is set to value 0 (weighting_1stChoice)
# the second choice project position in this array is set to value 1 (weighting_2ndChoice)
#etc

# Note: This allows for a different number of selections by each student
for s in range(num_students):
  for p  in range(len(projectChoices[s])):
    val = projectChoices[s][p]-1
    
    if p==0:
      costs[s, val] = weighting_1stChoice
    elif p==1:
      costs[s, val] = weighting_2ndChoice
    elif p==2:
      costs[s, val] = weighting_3rdChoice
    elif p==3:
      costs[s, val] = weighting_4thChoice
    elif p==4:
      costs[s, val] = weighting_5thChoice
    elif p==5:
      costs[s, val] = weighting_6thChoice
    elif p==6:
      costs[s, val] = weighting_7thChoice
    elif p==7:
      costs[s, val] = weighting_8thChoice


In [8]:
# We then create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
  print("ERROR!! no solver")

# The following code creates binary integer variables for the problem.

# x[i, j] is an array of 0-1 variables, which will be 1
# if student i is assigned to project j.
x = {}
for s in range(num_students):
    for p in range(totalNoProjectsOffered):
        x[s, p] = solver.IntVar(0, 1, '')

# The following code creates the constraints for the problem.
 
 # Each student is assigned to exactly 1 project.
for s in range(num_students):
    solver.Add(solver.Sum([x[s, p] for p in range(totalNoProjectsOffered)]) == 1)

# Each task is assigned to maximum number of students as stored in array 'maxNoStudentsPerProj'.
for p in range(totalNoProjectsOffered):
    solver.Add(solver.Sum([x[s, p] for s in range(num_students)]) <= maxNoStudentsPerProj[p])

# The following code creates the objective function for the problem.
objective_terms = []
for s in range(num_students):
    for p in range(totalNoProjectsOffered):
        objective_terms.append(costs[s][p] * x[s, p])
solver.Minimize(solver.Sum(objective_terms))

# The value of the objective function is the total cost over all variables that are assigned the value 1 by the solver.

# The following code invokes the solver.
status = solver.Solve()

 # The following code prints the solution to the problem.

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solver.Objective().Value()}\n\n')
    for s in range(num_students):
        for p in range(totalNoProjectsOffered):
            # Test if x[s,j] is 1 (with tolerance for floating point arithmetic).
            if x[s, p].solution_value() > 0.5:
              #print(f'STUDENT {s+1} (ID: {studentIDs[s]}) was assigned to project no. {p+1}.')
              
              if(costs[s][p] == weighting_1stChoice):
                  print(f'STUDENT {s+1} (ID: {studentIDs[s]}) was assigned to project no. {p+1}.   This was their: first choice') # + f' , x[i, j].solution_value(): {x[i, j].solution_value()}')
              elif(costs[s][p] == weighting_2ndChoice):
                  print(f'STUDENT {s+1} (ID: {studentIDs[s]}) was assigned to project no. {p+1}.   This was their: second choice')
              elif(costs[s][p] == weighting_3rdChoice):
                  print(f'STUDENT {s+1} (ID: {studentIDs[s]}) was assigned to project no. {p+1}.   This was their: third choice')
              elif(costs[s][p] == weighting_4thChoice):
                  print(f'STUDENT {s+1} (ID: {studentIDs[s]}) was assigned to project no. {p+1}.   This was their: fourth choice')
              elif(costs[s][p] == weighting_5thChoice):
                  print(f'STUDENT {s+1} (ID: {studentIDs[s]}) was assigned to project no. {p+1}.   This was their: fifth choice')
              elif(costs[s][p] == weighting_6thChoice):
                  print(f'STUDENT {s+1} (ID: {studentIDs[s]}) was assigned to project no. {p+1}.   This was their: sixth choice')
              elif(costs[s][p] == weighting_7thChoice):
                  print(f'STUDENT {s+1} (ID: {studentIDs[s]}) was assigned to project no. {p+1}.   This was their: seventh choice')
              elif(costs[s][p] == weighting_8thChoice):
                  print(f'STUDENT {s+1} (ID: {studentIDs[s]}) was assigned to project no. {p+1}.   This was their: eighth choice')

else:
    print('No solution found.')
    raise RuntimeError('SOMETHING HAS GONE WRONG: No solution found')

Total cost = 14.0


STUDENT 1 (ID: 1) was assigned to project no. 70.   This was their: second choice
STUDENT 2 (ID: 2) was assigned to project no. 21.   This was their: first choice
STUDENT 3 (ID: 3) was assigned to project no. 12.   This was their: first choice
STUDENT 4 (ID: 4) was assigned to project no. 13.   This was their: first choice
STUDENT 5 (ID: 5) was assigned to project no. 20.   This was their: first choice
STUDENT 6 (ID: 6) was assigned to project no. 60.   This was their: first choice
STUDENT 7 (ID: 7) was assigned to project no. 34.   This was their: first choice
STUDENT 8 (ID: 8) was assigned to project no. 39.   This was their: third choice
STUDENT 9 (ID: 9) was assigned to project no. 69.   This was their: first choice
STUDENT 10 (ID: 10) was assigned to project no. 4.   This was their: third choice
STUDENT 11 (ID: 11) was assigned to project no. 2.   This was their: first choice
STUDENT 12 (ID: 12) was assigned to project no. 32.   This was their: third choice
STU